In [ ]:
#default_exp extract_features

# Extract the image features

In [ ]:
#exporti
from fastcore.script import *
from fastai.vision.all import *
from fastai.callback.wandb import *
from self_supervised.augmentations import *
from self_supervised.layers import *
from self_supervised.vision.barlow_twins import *

# fixup depreciation
from kornia import augmentation as korniatfm
korniatfm.GaussianBlur = korniatfm.RandomGaussianBlur

In [ ]:
#export
@call_parse
def extract_features(
        path:Path, # path
        backbone:str    = 'resnet18', # backbone
        pretrained:bool = True, # pretrained
        size:int        = 224, # size
        projection_size = 2048, # projection_size
        lr:float        = 5e-4, # lr
        epochs:int      = 5, # epochs
        model_file:str  = None, # model .pth file
    ):
    "Pretrain a model on images in `path`."
    if model_file is None:
        model_path = path/'models'
        models = list(model_path.glob('*.pth'))
        if len(models) == 0:
            raise Exception(f"No models found in: {model_path}!")
        elif len(models) > 1:
            msg = f"Found multiple models in: {model_path}, please pass one of:"
            for model in models:
                msg += f"\n  --model_file {model}"
            raise Exception(msg)
        model_file = models[0]

    fastai_encoder = create_encoder('resnet18', pretrained=pretrained)
    model = create_barlow_twins_model(fastai_encoder, projection_size=projection_size, hidden_size=projection_size)

    dls = ImageDataLoaders.from_name_func(path, get_image_files(path), lambda x: False, item_tfms=Resize(size), valid_pct=0)
    fnames = get_image_files(path)
    test_dl = dls.test_dl(fnames)

    load_model(model_file, model, None, with_opt=False)
    if torch.cuda.is_available():
        model = model.cuda()
    
    features = []
    with torch.no_grad():
        for (dx,) in progress_bar(test_dl):
            features.append(model(dx).detach())
    raw_features = torch.cat(features)
    features = raw_features / torch.linalg.vector_norm(raw_features, dim=1, keepdims=True)

    with open(path/(Path(model_file).with_suffix('').name+'-features.pkl'), 'wb') as f:
        pickle.dump(dict(fnames = list(fnames), features = features), f)

In [ ]:
extract_features(Path('../women-in-tshirts-256/'), model_file='../women-in-tshirts-256/models/barlow-twins-resnet18-pretrained-224-5e-proj2048-lr0.5e-3.pth')